In [18]:
from bs4 import BeautifulSoup
import urllib3
import re
import time
from collections import defaultdict
import numpy as np
import tqdm
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from bs4 import BeautifulSoup 
import re
import requests

In [19]:
urlpage_4 = 'https://www.skysports.com/premier-league-table/2023'
def get_page(urlpage_4,element,html_class):
    # avoir la page en html
    req_5 = urllib3.PoolManager()
    res_5 = req_5.request('GET', urlpage_4)
    row_html_5 = BeautifulSoup(res_5.data, 'html.parser')
    
    # Renvoie les éléments correspondant à la classe HTML dans une liste
    PL19 = row_html_5.find_all(element , 
    class_= html_class)
    return(PL19)

PL19= str(get_page(urlpage_4, 'tr', 'row-body'))


In [20]:
list_team_20 = re.findall('<span class="team-name">(.*?)</span>', str(PL19))


In [21]:
def extract_team_20_stats(PL19, team):
    team= team.title()
    teams = re.findall('<span class="team-name">(.*?)</span>', 
    str(PL19))
    position= (list_team_20.index(team)+1)
    start = PL19.find(team)
    end = PL19.index("</tr>", start)
    team_data_20 = PL19[start:end]
    match_played= 38
    data = [int(s) for s in re.findall(r'<td.*?>(\d+)</td>', team_data_20)]
    points= data[0]
    wins= data [1]
    drawns= data [2]
    loses =data [3]
    goals_for = data [4]
    goals_against = data [5]
    team_stats20 = {'match_played': match_played,
    'position': position,'points': points,
                    'wins': wins,'loses': loses ,
                    'drawns':  drawns,'goals_for': goals_for,
        'goals_against':goals_against
    }
    return team_stats20

In [22]:
team_stats_20 = {}

In [23]:
for team in list_team_20:
    # Get team statistics with the previous function
    team_stats = extract_team_20_stats(PL19, team)
    # Convert team statistics to a dataframe
    team_stats_df = pd.DataFrame(team_stats, index=[0])
    # Create a team name variable in the dataframe
    team_stats_df['team'] = team
    team_stats_df['year'] = 2020
    # Let's add the dataframe to the dictionnary 
    ## (note that teams are like keys of this dictionnary)
    team_stats_20[team] = team_stats_df

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_premier_league_standings(year):
    url = f"https://www.skysports.com/premier-league-table/{year}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for {year}.")
        return None
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='standing-table__table')
    if table is None:
        print("Failed to find the standings table.")
        return None
    
    standings_data = []
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        team_name = columns[1].text.strip()
        matches_played = int(columns[2].text.strip())
        wins = int(columns[3].text.strip())
        draws = int(columns[4].text.strip())
        losses = int(columns[5].text.strip())
        goals_for = int(columns[6].text.strip())
        goals_against = int(columns[7].text.strip())
        goal_difference = int(columns[8].text.strip())
        points = int(columns[9].text.strip())
        
        standings_data.append({
            'Team': team_name,
            'Matches Played': matches_played,
            'Wins': wins,
            'Draws': draws,
            'Losses': losses,
            'Goals For': goals_for,
            'Goals Against': goals_against,
            'Goal Difference': goal_difference,
            'Points': points
        })
    
    return standings_data

def extract_team_20_stats(standings, year):
    team_stats_20 = {}
    for team_data in standings:
        team_name = team_data['Team']
        stats = extract_team_stats_single(team_data, year)
        team_stats_df = pd.DataFrame(stats, index=[0])
        team_stats_df['team'] = team_name
        team_stats_df['year'] = year
        team_stats_20[team_name] = team_stats_df
    
    if not team_stats_20:
        print("No data extracted.")
        return None
    
    return pd.concat(team_stats_20.values(), ignore_index=True)

def extract_team_stats_single(team_data, year):
    match_played = team_data['Matches Played']
    points = team_data['Points']
    wins = team_data['Wins']
    losses = team_data['Losses']
    draws = team_data['Draws']
    goals_for = team_data['Goals For']
    goals_against = team_data['Goals Against']
    
    team_stats = {
        'year': year,
        'team': None,  # Will be filled later
        'match_played': match_played,
        'position': None,  # Ensure to fill this
        'points': points,
        'wins': wins,
        'loses': losses,
        'drawns': draws,
        'goals_for': goals_for,
        'goals_against': goals_against
    }
    
    return team_stats

# Exemple d'utilisation
year = 2023
standings = scrape_premier_league_standings(year)
if standings:
    team_stats_20 = extract_team_20_stats(standings, year)
    print(team_stats_20)


    year                      team  match_played position  points  wins  \
0   2023                 Liverpool            29     None      67    20   
1   2023                   Arsenal            29     None      65    20   
2   2023           Manchester City            29     None      64    19   
3   2023               Aston Villa            30     None      59    18   
4   2023         Tottenham Hotspur            29     None      56    17   
5   2023         Manchester United            29     None      48    15   
6   2023           West Ham United            30     None      44    12   
7   2023          Newcastle United            29     None      43    13   
8   2023  Brighton and Hove Albion            29     None      42    11   
9   2023   Wolverhampton Wanderers            29     None      41    12   
10  2023                   Chelsea            28     None      40    11   
11  2023                    Fulham            30     None      39    11   
12  2023               Bo

In [25]:
url_7 = "https://sportune.20minutes.fr/sport-business/football/les-budgets-des-clubs-de-la-premier-league-2023-2024-312241/2"

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_premier_league_budgets(url):
    # Récupérer le contenu de la page web
    response = requests.get(url)
    
    # Vérifier si la requête a réussi
    if response.status_code != 200:
        print(f"Failed to retrieve data from {url}.")
        return None
    
    # Parser le contenu HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Trouver la table contenant les données de budget des clubs
    table = soup.find('table', class_='tabla-comparativa-equipos')
    
    # Vérifier si la table a été trouvée
    if table is None:
        print("Failed to find the table.")
        return None
    
    # Extraire les données de la table
    rows = table.find_all('tr')
    data = []
    for row in rows[1:]:
        cells = row.find_all('td')
        club = cells[0].text.strip()
        budget = cells[1].text.strip()
        data.append([club, budget])
    
    # Créer un DataFrame pandas à partir des données extraites
    df = pd.DataFrame(data, columns=['Club', 'Budget'])
    
    return df

# URL de la page des budgets des clubs de la Premier League
url = "https://sportune.20minutes.fr/sport-business/football/les-budgets-des-clubs-de-la-premier-league-2023-2024-312241/2"

# Appel de la fonction et affichage des données
budgets_df = scrape_premier_league_budgets(url)
print(budgets_df)



Failed to find the table.
None


In [27]:
import pandas as pd
from bs4 import BeautifulSoup 
import re
import requests

def scrape_premier_league_budgets(url):
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
    
    res = requests.get(url, headers=user_agent)
    if res.status_code != 200:
        print("Failed to retrieve data.")
        return None
    
    content = res.text
    soup = BeautifulSoup(content, "lxml")
    contents = re.findall('<tbody>.*?\n</tbody>\n</table>', str(soup), re.DOTALL)
    
    all_headers = []
    for html_content in contents:
        html_soup = BeautifulSoup(html_content, 'html.parser')    
        headers = html_soup.find_all("th")    
        all_headers.extend(headers)
    
    Titles = [i.text for i in all_headers]
    df = pd.DataFrame(columns=Titles)
    
    all_rows = []
    for html_content in contents:
        html_soup = BeautifulSoup(html_content, 'html.parser')
        rows = html_soup.find_all("tr")
        rows_data = []
        for row in rows:
            cells = row.find_all("td")
            cell_data = [cell.get_text(strip=True) for cell in cells]
            rows_data.append(cell_data)
        all_rows.extend(rows_data)
    
    if all_rows:
        all_rows.pop(-1)
    
    df = pd.concat([df, pd.DataFrame(all_rows, columns=Titles)], ignore_index=True)
    return df

# Exemple d'utilisation
url = "https://sportune.20minutes.fr/sport-business/football/les-budgets-des-clubs-de-la-premier-league-2023-2024-312241/2"
budgets_df = scrape_premier_league_budgets(url)
print(budgets_df)


                 Club  Budget
0                None    None
1     Manchester City  800 M€
2   Manchester United  720 M€
3           Liverpool  690 M€
4             Chelsea  600 M€
5           Tottenham  550 M€
6             Arsenal  525 M€
7           Newcastle  305 M€
8            West Ham  290 M€
9             Everton  230 M€
10        Aston Villa  220 M€
11           Brighton  200 M€
12     Crystal Palace  200 M€
13      Wolverhampton  185 M€
14          Brentford  175 M€
15        Bournemouth  160 M€
16             Fulham  155 M€
17   Sheffield United  145 M€
18            Burnley  125 M€
19  Nottingham Forest  120 M€
20         Luton Town   90 M€


In [28]:
# Supposons que vous avez déjà les DataFrames `budgets_df` et `team_stats_20`
# Remplacer les noms d'équipes dans team_stats_20
team_stats_20.replace({'Tottenham Hotspur': 'Tottenham',
                       'Nottingham Forest **': 'Nottingham Forest',
                       'Everton *': 'Everton',
                       'West Ham United': 'West Ham',
                       'Wolverhampton Wanderers': 'Wolverhampton',
                       'Brighton and Hove Albion': 'Brighton',
                       'Newcastle United': 'Newcastle'}, inplace=True)

# Fusionner les deux DataFrames en utilisant la colonne "team" comme clé de fusion
merged_df = pd.merge(budgets_df, team_stats_20, left_on='Club', right_on='team', how='inner')

# Afficher le DataFrame fusionné
print(merged_df)






                 Club  Budget  year               team  match_played position  \
0     Manchester City  800 M€  2023    Manchester City            29     None   
1   Manchester United  720 M€  2023  Manchester United            29     None   
2           Liverpool  690 M€  2023          Liverpool            29     None   
3             Chelsea  600 M€  2023            Chelsea            28     None   
4           Tottenham  550 M€  2023          Tottenham            29     None   
5             Arsenal  525 M€  2023            Arsenal            29     None   
6           Newcastle  305 M€  2023          Newcastle            29     None   
7            West Ham  290 M€  2023           West Ham            30     None   
8             Everton  230 M€  2023            Everton            29     None   
9         Aston Villa  220 M€  2023        Aston Villa            30     None   
10           Brighton  200 M€  2023           Brighton            29     None   
11     Crystal Palace  200 M

In [29]:
# Exporter le DataFrame fusionné vers un fichier CSV
merged_df.to_csv('merged_2.csv', index=False)



In [30]:
# Supprimer la colonne 'Position' du DataFrame merged_df
merged_df.drop('position', axis=1, inplace=True)


In [31]:
print (merged_df)

                 Club  Budget  year               team  match_played  points  \
0     Manchester City  800 M€  2023    Manchester City            29      64   
1   Manchester United  720 M€  2023  Manchester United            29      48   
2           Liverpool  690 M€  2023          Liverpool            29      67   
3             Chelsea  600 M€  2023            Chelsea            28      40   
4           Tottenham  550 M€  2023          Tottenham            29      56   
5             Arsenal  525 M€  2023            Arsenal            29      65   
6           Newcastle  305 M€  2023          Newcastle            29      43   
7            West Ham  290 M€  2023           West Ham            30      44   
8             Everton  230 M€  2023            Everton            29      25   
9         Aston Villa  220 M€  2023        Aston Villa            30      59   
10           Brighton  200 M€  2023           Brighton            29      42   
11     Crystal Palace  200 M€  2023     

In [32]:
# Supprimer les caractères indésirables et convertir en valeurs numériques
merged_df['Budget'] = pd.to_numeric(merged_df['Budget'].str.replace('M€', '')) * 1000000

# Afficher le DataFrame mis à jour
print(merged_df)


                 Club     Budget  year               team  match_played  \
0     Manchester City  800000000  2023    Manchester City            29   
1   Manchester United  720000000  2023  Manchester United            29   
2           Liverpool  690000000  2023          Liverpool            29   
3             Chelsea  600000000  2023            Chelsea            28   
4           Tottenham  550000000  2023          Tottenham            29   
5             Arsenal  525000000  2023            Arsenal            29   
6           Newcastle  305000000  2023          Newcastle            29   
7            West Ham  290000000  2023           West Ham            30   
8             Everton  230000000  2023            Everton            29   
9         Aston Villa  220000000  2023        Aston Villa            30   
10           Brighton  200000000  2023           Brighton            29   
11     Crystal Palace  200000000  2023     Crystal Palace            29   
12      Wolverhampton  18

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assurez-vous que votre DataFrame contient la colonne "Budget"
# Si vous n'avez pas déjà fusionné les DataFrames merged_df et budgets_df, faites-le maintenant

# Définissez votre ensemble de caractéristiques (X) en incluant les variables nécessaires
X = merged_df[['wins', 'loses', 'drawns', 'goals_for', 'goals_against', 'Budget']]

# Définissez votre variable cible (y)
y = merged_df['points']

# Divisez vos données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créez un objet de modèle de régression linéaire
model = LinearRegression()

# Entraînez le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# Évaluez les performances du modèle sur les données de test
score = model.score(X_test, y_test)
print("Coefficient de détermination R^2 :", score)


Coefficient de détermination R^2 : 0.9814363461799384


In [34]:
import statsmodels.api as sm

# Variables indépendantes (caractéristiques)
X = merged_df[['wins', 'loses', 'drawns', 'goals_for', 'goals_against', 'Budget']]

# Variable dépendante (cible)
y = merged_df['points']

# Ajouter une constante aux variables indépendantes
X = sm.add_constant(X)

# Créer le modèle de régression de Poisson
poisson_model = sm.GLM(y, X, family=sm.families.Poisson())

# Ajuster le modèle aux données
poisson_results = poisson_model.fit()

# Afficher un résumé du modèle
print(poisson_results.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                 points   No. Observations:                   20
Model:                            GLM   Df Residuals:                       13
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -57.056
Date:                Tue, 02 Apr 2024   Deviance:                       5.0608
Time:                        22:38:44   Pearson chi2:                     4.89
No. Iterations:                     4   Pseudo R-squ. (CS):             0.9976
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.0778      2.523      1.617

In [35]:
# Nombre de matchs à prédire
nb_matchs = 38

# Coefficients du modèle
coefficients = poisson_results.params

# Prédictions des points pour chaque équipe
predicted_points = poisson_results.predict(X)

# Calcul des points supplémentaires pour chaque équipe en fonction des résultats de matchs prévus
predicted_wins_points = coefficients['wins'] * nb_matchs * 3
predicted_draws_points = coefficients['drawns'] * nb_matchs
predicted_losses_points = 0  # Les défaites ne contribuent pas aux points

# Ajouter les points supplémentaires aux prédictions de points
predicted_points += predicted_wins_points + predicted_draws_points + predicted_losses_points

# Classer les équipes en fonction des points prédits
predicted_points_ranking = predicted_points.sort_values(ascending=False)

# Afficher le classement des équipes prédites
print(predicted_points_ranking)


2     73.769824
5     70.359282
0     66.955858
9     63.822154
4     59.156821
1     46.528397
6     43.236282
7     43.102749
10    41.865554
12    41.797085
3     39.896623
14    39.231482
15    37.186615
8     30.699568
11    30.672962
13    27.305759
18    26.493192
19    24.620622
16    23.218271
17    22.630376
dtype: float64


In [36]:
# Concaténer les noms des équipes avec les prédictions de points
predicted_points_with_teams = pd.concat([merged_df['team'], predicted_points], axis=1)
predicted_points_with_teams.columns = ['Team', 'Predicted Points']

# Classer les équipes en fonction des points prédits
predicted_points_ranking = predicted_points_with_teams.sort_values(by='Predicted Points', ascending=False)
predicted_points_with_teams['Predicted Points'] = predicted_points_with_teams['Predicted Points'].astype(int)

# Afficher le classement des équipes prédites avec les noms des équipes
print(predicted_points_ranking)



                 Team  Predicted Points
2           Liverpool         73.769824
5             Arsenal         70.359282
0     Manchester City         66.955858
9         Aston Villa         63.822154
4           Tottenham         59.156821
1   Manchester United         46.528397
6           Newcastle         43.236282
7            West Ham         43.102749
10           Brighton         41.865554
12      Wolverhampton         41.797085
3             Chelsea         39.896623
14        Bournemouth         39.231482
15             Fulham         37.186615
8             Everton         30.699568
11     Crystal Palace         30.672962
13          Brentford         27.305759
18  Nottingham Forest         26.493192
19         Luton Town         24.620622
16   Sheffield United         23.218271
17            Burnley         22.630376


In [37]:
predicted_points_with_teams['Predicted Points'] = predicted_points_with_teams['Predicted Points'].astype(int)

# Afficher le classement des équipes prédites avec les noms des équipes
print(predicted_points_ranking)

                 Team  Predicted Points
2           Liverpool         73.769824
5             Arsenal         70.359282
0     Manchester City         66.955858
9         Aston Villa         63.822154
4           Tottenham         59.156821
1   Manchester United         46.528397
6           Newcastle         43.236282
7            West Ham         43.102749
10           Brighton         41.865554
12      Wolverhampton         41.797085
3             Chelsea         39.896623
14        Bournemouth         39.231482
15             Fulham         37.186615
8             Everton         30.699568
11     Crystal Palace         30.672962
13          Brentford         27.305759
18  Nottingham Forest         26.493192
19         Luton Town         24.620622
16   Sheffield United         23.218271
17            Burnley         22.630376


In [38]:
# Convertir les points prédits en entiers
predicted_points_with_teams['Predicted Points'] = predicted_points_with_teams['Predicted Points'].astype(int)
